# POI Registry Viewer

Visualizes composite Points of Interest (POIs) aggregated from all SMC concepts.
Each POI is a merged zone from overlapping FVGs, liquidity levels, and session levels.
Color intensity reflects POI strength score.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import plotly.graph_objects as go

from data.loader import load_instrument
from data.resampler import resample
from visualization.chart import candlestick_chart
from concepts.fractals import detect_swings, get_swing_points
from concepts.fvg import detect_fvg, track_fvg_lifecycle
from concepts.liquidity import detect_equal_levels, detect_session_levels
from concepts.registry import build_poi_registry

print('All imports OK')

In [2]:
# Load and resample data
df_1m = load_instrument('NAS100')
df_15m = resample(df_1m, '15m')
print(f'Loaded {len(df_15m):,} bars of 15m data')

# View slice
SLICE_START = 500
SLICE_END = 1000
df_view = df_15m.iloc[SLICE_START:SLICE_END].copy()
print(f'View: {len(df_view)} bars')

FVG_MIN_GAP_PCT = 0.0003

Loaded 73,953 bars of 15m data
View: 500 bars


## Detect All Concepts

In [ ]:
# Detect all concepts on the view slice
fvgs = detect_fvg(df_view, min_gap_pct=FVG_MIN_GAP_PCT)
lifecycle = track_fvg_lifecycle(df_view, fvgs, mitigation_mode='close', max_age_bars=192)
eq_levels = detect_equal_levels(df_view, swing_length=5)
session_levels = detect_session_levels(df_view, level_type='daily')

print(f'FVGs: {len(fvgs)}')
print(f'Liquidity levels: {len(eq_levels)}, Session levels: {len(session_levels)}')

## Build POI Registry

In [ ]:
pois = build_poi_registry(
    fvgs, eq_levels, session_levels,
    fvg_lifecycle=lifecycle,
    timeframe='15m',
)

print(f'Total POIs: {len(pois)}')
if len(pois) > 0:
    bullish = pois[pois['direction'] == 1]
    bearish = pois[pois['direction'] == -1]
    print(f'  Bullish (demand): {len(bullish)}')
    print(f'  Bearish (supply): {len(bearish)}')
    print(f'  Score range: {pois["score"].min():.1f} - {pois["score"].max():.1f}')
    multi = pois[pois['component_count'] >= 2]
    print(f'  Confluence POIs (2+ components): {len(multi)}')
    print(f'  Max components in single POI: {pois["component_count"].max()}')
    print()
    print('Top 10 POIs by score:')
    display_cols = ['direction', 'top', 'bottom', 'score', 'component_count', 'status']
    print(pois[display_cols].head(10).to_string(index=False))

## POI Visualization

Rectangles for each POI, color intensity proportional to score.
Bullish POIs = green, Bearish POIs = red.

In [5]:
fig = candlestick_chart(df_view, title='NAS100 15m - POI Registry (All Concepts)', height=700)

if len(pois) > 0:
    max_score = pois['score'].max()
    x0 = df_view['time'].iloc[0] if 'time' in df_view.columns else df_view.index[0]
    x1 = df_view['time'].iloc[-1] if 'time' in df_view.columns else df_view.index[-1]

    for _, poi in pois.iterrows():
        # Opacity scales with score (min 0.08, max 0.35)
        opacity = 0.08 + 0.27 * (poi['score'] / max_score) if max_score > 0 else 0.15
        border_opacity = min(opacity * 3, 0.9)
        # Border width scales with component count
        border_width = min(1 + poi['component_count'], 4)

        if poi['direction'] == 1:
            fill = f'rgba(0,180,0,{opacity:.2f})'
            border = f'rgba(0,180,0,{border_opacity:.2f})'
        else:
            fill = f'rgba(180,0,0,{opacity:.2f})'
            border = f'rgba(180,0,0,{border_opacity:.2f})'

        fig.add_shape(
            type='rect', x0=x0, x1=x1,
            y0=poi['bottom'], y1=poi['top'],
            fillcolor=fill,
            line=dict(color=border, width=border_width),
        )

        # Label for high-score POIs
        if poi['score'] >= max_score * 0.5:
            label_y = (poi['top'] + poi['bottom']) / 2
            fig.add_annotation(
                x=x0, y=label_y,
                text=f"S={poi['score']:.0f} C={poi['component_count']}",
                showarrow=False, font=dict(size=9, color='white'),
                bgcolor='rgba(0,0,0,0.5)', borderpad=2,
            )

fig.update_layout(legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01))
fig.show()

## Score Distribution

In [6]:
if len(pois) > 0:
    import plotly.express as px

    fig_hist = px.histogram(
        pois, x='score', nbins=20,
        title='POI Score Distribution',
        labels={'score': 'POI Score', 'count': 'Count'},
    )
    fig_hist.show()

    fig_comp = px.histogram(
        pois, x='component_count', nbins=10,
        title='POI Component Count Distribution',
        labels={'component_count': 'Components', 'count': 'Count'},
    )
    fig_comp.show()
else:
    print('No POIs to analyze')

## Confluence POIs Only

Show only POIs with 2+ overlapping components (stronger signals).

In [ ]:
# Shorter slice for cleaner view
df_short = df_15m.iloc[SLICE_START:SLICE_START+200].copy()

# Detect concepts on short slice
fvg_short = detect_fvg(df_short, min_gap_pct=FVG_MIN_GAP_PCT)
lc_short = track_fvg_lifecycle(df_short, fvg_short, mitigation_mode='close', max_age_bars=192)
eq_short = detect_equal_levels(df_short, swing_length=5)
ss_short = detect_session_levels(df_short, level_type='daily')

pois_short = build_poi_registry(
    fvg_short, eq_short, ss_short,
    fvg_lifecycle=lc_short, timeframe='15m',
)

confluence = pois_short[pois_short['component_count'] >= 2] if len(pois_short) > 0 else pois_short
print(f'Confluence POIs: {len(confluence)} / {len(pois_short)} total')

fig = candlestick_chart(df_short, title='NAS100 15m - Confluence POIs Only (2+ components)', height=600)

if len(confluence) > 0:
    x0 = df_short['time'].iloc[0] if 'time' in df_short.columns else df_short.index[0]
    x1 = df_short['time'].iloc[-1] if 'time' in df_short.columns else df_short.index[-1]
    max_score = confluence['score'].max()

    for _, poi in confluence.iterrows():
        opacity = 0.12 + 0.25 * (poi['score'] / max_score) if max_score > 0 else 0.2
        border_width = min(1 + poi['component_count'], 4)

        if poi['direction'] == 1:
            fill = f'rgba(0,200,0,{opacity:.2f})'
            border = f'rgba(0,200,0,0.7)'
        else:
            fill = f'rgba(200,0,0,{opacity:.2f})'
            border = f'rgba(200,0,0,0.7)'

        fig.add_shape(
            type='rect', x0=x0, x1=x1,
            y0=poi['bottom'], y1=poi['top'],
            fillcolor=fill,
            line=dict(color=border, width=border_width),
        )

        # Label
        label_y = (poi['top'] + poi['bottom']) / 2
        dir_label = 'BUL' if poi['direction'] == 1 else 'BER'
        fig.add_annotation(
            x=x0, y=label_y,
            text=f"{dir_label} S={poi['score']:.0f} C={poi['component_count']}",
            showarrow=False, font=dict(size=9, color='white'),
            bgcolor='rgba(0,0,0,0.6)', borderpad=2,
        )

fig.show()